In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


In [2]:
# data
train_data = pd.read_csv("C:/Users/Luke/OneDrive/Research/sg_data/sg_hole_train.csv")

test_data = pd.read_csv("C:/Users/Luke/OneDrive/Research/sg_data/sg_hole_test.csv")

In [3]:
# nodsr estimates
nodsr_p_ests = pd.read_csv('model_data/nodsr_p_ests_0814_train.csv')
nodsr_c_ests = pd.read_csv('model_data/nodsr_c_ests_0814_train.csv')

# dsr estimates
dsr_p_ests = pd.read_csv('model_data/dsr_p_ests_0814_train.csv')
dsr_c_ests = pd.read_csv('model_data/dsr_c_ests_0814_train.csv')
dsr_d_ests = pd.read_csv('model_data/dsr_d_ests_0814_train.csv')

In [4]:
nodsr_p_ests_avg = nodsr_p_ests.groupby(['plyr_year','p_y_idx']).mean().reset_index()
nodsr_c_ests_avg = nodsr_c_ests.groupby(['course_name','c_idx']).mean().reset_index()


dsr_p_ests_avg = dsr_p_ests.groupby(['plyr_year','p_y_idx']).mean().reset_index()
dsr_c_ests_avg = dsr_c_ests.groupby(['course_name','c_idx']).mean().reset_index()
dsr_d_ests_avg = dsr_d_ests.groupby(['course_name','c_idx']).mean().reset_index()

In [5]:
print('train shape: ', train_data.shape)
train_data_w_ests = pd.merge(train_data, nodsr_p_ests_avg, on = 'p_y_idx', suffixes = ['','_nodsr_plyr'])
print('train shape: ', train_data_w_ests.shape)
train_data_w_ests = pd.merge(train_data_w_ests, nodsr_c_ests_avg, on = 'c_idx', suffixes = ['','_nodsr_course'])

print('train shape: ', train_data_w_ests.shape)
train_data_w_ests = pd.merge(train_data_w_ests, dsr_p_ests_avg, on = 'p_y_idx', suffixes = ['','_dsr_plyr'])
print('train shape: ', train_data_w_ests.shape)
train_data_w_ests = pd.merge(train_data_w_ests, dsr_c_ests_avg, on = 'c_idx', suffixes = ['','_dsr_course'])
train_data_w_ests = pd.merge(train_data_w_ests, dsr_d_ests_avg, on = 'c_idx', suffixes = ['','_dsr_course'])

train shape:  (70669, 27)
train shape:  (70669, 33)
train shape:  (70669, 39)
train shape:  (70669, 45)


In [6]:
nodsr_p_ests_avg

,plyr_year,p_y_idx,sg_drive,sg_appr,sg_short,sg_putt,num_rounds
0,"(""Adam D'Amario"", 2018)",19,-1.333300,-3.893122,-0.693278,-1.940855,2.0
1,"(""Bronson La'Cassie"", 2014)",422,-0.050633,-0.602725,0.149570,-0.133639,36.0
2,"(""Sam O'Dell"", 2015)",2375,-0.784407,-0.211334,-0.155453,0.097794,2.0
3,"(""Sam O'Dell"", 2018)",2376,0.577920,-1.653313,-2.091368,-1.190574,2.0
4,"(""Sean O'Hair"", 2014)",2447,0.109332,-0.532446,0.019670,0.064983,67.0
...,...,...,...,...,...,...,...
2429,"('Zack Vervaecke', 2015)",2906,-0.922620,-2.254541,-0.250326,-0.172434,2.0
2430,"('Zechariah Potter', 2015)",2907,-2.521448,-1.513633,0.588002,0.128880,2.0
2431,"('Zecheng Dou', 2018)",2908,-0.155140,-0.361587,-0.204344,-0.724253,36.0
2432,"('Ángel Cabrera', 2018)",2909,0.183059,-0.748187,-0.546637,-0.445941,7.0


In [7]:
nodsr_p_ests_avg['plyr_year_new'] = nodsr_p_ests_avg['plyr_year'].apply(lambda row: row[:-5] + '2019)' if row[-5:] == '2018)' else row)
dsr_p_ests_avg['plyr_year_new'] = dsr_p_ests_avg['plyr_year'].apply(lambda row: row[:-5] + '2019)' if row[-5:] == '2018)' else row)

print('test shape: ', test_data.shape)
test_data_w_ests = pd.merge(test_data, nodsr_p_ests_avg, how = 'left', left_on = 'plyr_year', right_on = 'plyr_year_new', suffixes = ['','_nodsr_plyr'])
print('test shape: ', test_data_w_ests.shape)
test_data_w_ests = pd.merge(test_data_w_ests, nodsr_c_ests_avg, how = 'left', on = 'c_idx', suffixes = ['','_nodsr_course'])

print('test shape: ', test_data_w_ests.shape)
test_data_w_ests = pd.merge(test_data_w_ests, dsr_p_ests_avg,  how = 'left', left_on = 'plyr_year', right_on = 'plyr_year_new', suffixes = ['','_dsr_plyr'])
print('test shape: ', test_data_w_ests.shape)
test_data_w_ests = pd.merge(test_data_w_ests, dsr_c_ests_avg, how = 'left', on = 'c_idx', suffixes = ['','_dsr_course'])
test_data_w_ests = pd.merge(test_data_w_ests, dsr_d_ests_avg, how = 'left', on = 'c_idx', suffixes = ['','_dsr_course'])

test shape:  (12952, 27)
test shape:  (12952, 35)
test shape:  (12952, 41)
test shape:  (12952, 49)


In [8]:
train_data_w_ests['nodsr_drive_est'] = train_data_w_ests['sg_drive'] + train_data_w_ests['sg_drive_nodsr_course']
train_data_w_ests['nodsr_appr_est'] = train_data_w_ests['sg_appr'] + train_data_w_ests['sg_appr_nodsr_course']
train_data_w_ests['nodsr_short_est'] = train_data_w_ests['sg_short'] + train_data_w_ests['sg_short_nodsr_course']
train_data_w_ests['nodsr_putt_est'] = train_data_w_ests['sg_putt'] + train_data_w_ests['sg_putt_nodsr_course']
train_data_w_ests['nodsr_total_est'] = train_data_w_ests['nodsr_drive_est'] + train_data_w_ests['nodsr_appr_est'] + train_data_w_ests['nodsr_short_est'] + train_data_w_ests['nodsr_putt_est']

train_data_w_ests['dsr_drive_est'] = train_data_w_ests['sg_drive_dsr_plyr']*train_data_w_ests['dsr_drive'] + train_data_w_ests['sg_drive_dsr_course']
train_data_w_ests['dsr_appr_est'] = train_data_w_ests['sg_appr_dsr_plyr']*train_data_w_ests['dsr_appr'] + train_data_w_ests['sg_appr_dsr_course']
train_data_w_ests['dsr_short_est'] = train_data_w_ests['sg_short_dsr_plyr']*train_data_w_ests['dsr_short'] + train_data_w_ests['sg_short_dsr_course']
train_data_w_ests['dsr_putt_est'] = train_data_w_ests['sg_putt_dsr_plyr']*train_data_w_ests['dsr_putt'] + train_data_w_ests['sg_putt_dsr_course']
train_data_w_ests['dsr_total_est'] = train_data_w_ests['dsr_drive_est'] + train_data_w_ests['dsr_appr_est'] + train_data_w_ests['dsr_short_est'] + train_data_w_ests['dsr_putt_est']

train_data_w_ests['SG_total'] = train_data_w_ests['SG_drive'] + train_data_w_ests['SG_appr'] + train_data_w_ests['SG_short'] + train_data_w_ests['SG_putt']


In [9]:
test_data_w_ests['nodsr_drive_est'] = test_data_w_ests['sg_drive'] + test_data_w_ests['sg_drive_nodsr_course']
test_data_w_ests['nodsr_appr_est'] = test_data_w_ests['sg_appr'] + test_data_w_ests['sg_appr_nodsr_course']
test_data_w_ests['nodsr_short_est'] = test_data_w_ests['sg_short'] + test_data_w_ests['sg_short_nodsr_course']
test_data_w_ests['nodsr_putt_est'] = test_data_w_ests['sg_putt'] + test_data_w_ests['sg_putt_nodsr_course']
test_data_w_ests['nodsr_total_est'] = test_data_w_ests['nodsr_drive_est'] + test_data_w_ests['nodsr_appr_est'] + test_data_w_ests['nodsr_short_est'] + test_data_w_ests['nodsr_putt_est']

test_data_w_ests['dsr_drive_est'] = test_data_w_ests['sg_drive_dsr_plyr']*test_data_w_ests['dsr_drive'] + test_data_w_ests['sg_drive_dsr_course']
test_data_w_ests['dsr_appr_est'] = test_data_w_ests['sg_appr_dsr_plyr']*test_data_w_ests['dsr_appr'] + test_data_w_ests['sg_appr_dsr_course']
test_data_w_ests['dsr_short_est'] = test_data_w_ests['sg_short_dsr_plyr']*test_data_w_ests['dsr_short'] + test_data_w_ests['sg_short_dsr_course']
test_data_w_ests['dsr_putt_est'] = test_data_w_ests['sg_putt_dsr_plyr']*test_data_w_ests['dsr_putt'] + test_data_w_ests['sg_putt_dsr_course']
test_data_w_ests['dsr_total_est'] = test_data_w_ests['dsr_drive_est'] + test_data_w_ests['dsr_appr_est'] + test_data_w_ests['dsr_short_est'] + test_data_w_ests['dsr_putt_est']

test_data_w_ests['SG_total'] = test_data_w_ests['SG_drive'] + test_data_w_ests['SG_appr'] + test_data_w_ests['SG_short'] + test_data_w_ests['SG_putt']


In [10]:
print('TRAINING DATA')
print('NODSR TOTAL: ', mean_squared_error(train_data_w_ests['nodsr_total_est'], train_data_w_ests['SG_total'], squared = False))
print('DSR TOTAL: ', mean_squared_error(train_data_w_ests['dsr_total_est'], train_data_w_ests['SG_total'], squared = False))
print()
print('NODSR DRIVE: ', mean_squared_error(train_data_w_ests['nodsr_drive_est'], train_data_w_ests['SG_drive'], squared = False))
print('DSR DRIVE: ', mean_squared_error(train_data_w_ests['dsr_drive_est'], train_data_w_ests['SG_drive'], squared = False))
print()
print('NODSR APPR: ', mean_squared_error(train_data_w_ests['nodsr_appr_est'], train_data_w_ests['SG_appr'], squared = False))
print('DSR APPR: ', mean_squared_error(train_data_w_ests['dsr_appr_est'], train_data_w_ests['SG_appr'], squared = False))

print()
print('NODSR SHORT: ', mean_squared_error(train_data_w_ests['nodsr_short_est'], train_data_w_ests['SG_short'], squared = False))
print('DSR SHORT: ', mean_squared_error(train_data_w_ests['dsr_short_est'], train_data_w_ests['SG_short'], squared = False))
print()
print('NODSR PUTT: ', mean_squared_error(train_data_w_ests['nodsr_putt_est'], train_data_w_ests['SG_putt'], squared = False))
print('DSR PUTT: ', mean_squared_error(train_data_w_ests['dsr_putt_est'], train_data_w_ests['SG_putt'], squared = False))


TRAINING DATA
NODSR TOTAL:  2.745254099819849
DSR TOTAL:  2.7428838760334813

NODSR DRIVE:  1.0818811351596005
DSR DRIVE:  1.081299758586981

NODSR APPR:  1.5751761586227915
DSR APPR:  1.573544598977392

NODSR SHORT:  1.0791059492612973
DSR SHORT:  1.0780058651518838

NODSR PUTT:  1.6749732633970051
DSR PUTT:  1.673580843393714


In [11]:
test_data_w_ests = test_data_w_ests.dropna()

In [12]:
print('TESTING DATA')
print('NODSR TOTAL: ', mean_squared_error(test_data_w_ests['nodsr_total_est'], test_data_w_ests['SG_total'], squared = False))
print('DSR TOTAL: ', mean_squared_error(test_data_w_ests['dsr_total_est'], test_data_w_ests['SG_total'], squared = False))
print()
print('NODSR DRIVE: ', mean_squared_error(test_data_w_ests['nodsr_drive_est'], test_data_w_ests['SG_drive'], squared = False))
print('DSR DRIVE: ', mean_squared_error(test_data_w_ests['dsr_drive_est'], test_data_w_ests['SG_drive'], squared = False))
print()
print('NODSR APPR: ', mean_squared_error(test_data_w_ests['nodsr_appr_est'], test_data_w_ests['SG_appr'], squared = False))
print('DSR APPR: ', mean_squared_error(test_data_w_ests['dsr_appr_est'], test_data_w_ests['SG_appr'], squared = False))

print()
print('NODSR SHORT: ', mean_squared_error(test_data_w_ests['nodsr_short_est'], test_data_w_ests['SG_short'], squared = False))
print('DSR SHORT: ', mean_squared_error(test_data_w_ests['dsr_short_est'], test_data_w_ests['SG_short'], squared = False))
print()
print('NODSR PUTT: ', mean_squared_error(test_data_w_ests['nodsr_putt_est'], test_data_w_ests['SG_putt'], squared = False))
print('DSR PUTT: ', mean_squared_error(test_data_w_ests['dsr_putt_est'], test_data_w_ests['SG_putt'], squared = False))


TESTING DATA
NODSR TOTAL:  2.909703153896128
DSR TOTAL:  2.917754184688774

NODSR DRIVE:  1.1686584017895498
DSR DRIVE:  1.1685132868240624

NODSR APPR:  1.5963319531946805
DSR APPR:  1.6033147151529041

NODSR SHORT:  1.1073499292018807
DSR SHORT:  1.1129201011198704

NODSR PUTT:  1.7736115019969927
DSR PUTT:  1.7722333023412236
